# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark
findspark.init('/home/vicky/spark-3.0.0-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('consulting').getOrCreate()

In [4]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [25]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [22]:
data_f = data.na.drop()

In [23]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [26]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [27]:
output = assembler.transform(data)

In [28]:
final_data = output.select('features','churn')

In [29]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [30]:
from pyspark.ml.classification import LogisticRegression

In [31]:
lr_churn = LogisticRegression(labelCol='churn')

In [32]:
fitted_churn_model = lr_churn.fit(train_churn)

In [33]:
training_sum = fitted_churn_model.summary

In [34]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                624|                624|
|   mean|0.15705128205128205| 0.1201923076923077|
| stddev|0.36414100940913813|0.32544715978037386|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [35]:
#evaluation
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [36]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [37]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.96171350679426...|[0.99304775072829...|       0.0|
|[25.0,9672.03,0.0...|    0|[4.92214171370366...|[0.99276915106950...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.85608776827577...|[0.70184262948408...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.60040068957640...|[0.99864202354625...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.86035337288698...|[0.99715786444947...|       0.0|
|[28.0,11245.38,0....|    0|[4.03121725893413...|[0.98255695413053...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.50974075606763...|[0.98911840004481...|       0.0|
|[29.0,11274.46,1....|    0|[4.72201252463623...|[0.99118120845913...|       0.0|
|[29.0,12711.15,0....|    0|[5.59061355786095...|[0.99628114459696...|       0.0|
|[29.0,13240.01,

In [38]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [39]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [40]:
auc

0.7395260989010989

In [41]:
#Predicting new data
final_lr_model = lr_churn.fit(final_data)

In [42]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [43]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [44]:
test_new_customers = assembler.transform(new_customers)

In [45]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [46]:
final_results = final_lr_model.transform(test_new_customers)

In [47]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

